<h2> Load Library <h2>

In [1]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from keras import backend as K
K.set_image_dim_ordering('th')
import pickle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

Using TensorFlow backend.


<h2> Function For ganti indx to String Names <h2>

In [2]:
def from_class_to_label(kelas):
    data_path = os.path.expanduser('data_gambar_sklearn')
    data_dir_list = sorted(os.listdir(data_path))

    data_nama=[]
    for dataset in data_dir_list:
        img_list=os.listdir(data_path+'/'+ dataset)
        data_nama.append(dataset)
    
    list_labels = data_nama
    
    for i in list_labels:
        if kelas==list_labels.index(i):
            return i

print("Fungsi berhasil di load")

Fungsi berhasil di load


<h2> Display a video with Opencv dan Python<h2>

In [3]:
def show_video():
        video_capture = cv2.VideoCapture(0)
        nama_muka = []
        while True:
            ret, frame = video_capture.read()
            inImg = np.array(frame)
            outImg, nama_muka = process_image(inImg)
            cv2.imshow('Mantul', outImg)

            # When everything is done, release the capture on pressing 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                video_capture.release()
                cv2.destroyAllWindows()
                return

<h2> function for resize <h2>

In [4]:
def face1(face):
    num_channel = 1
    face_resized = cv2.resize(face, (112, 92))
    test_image = np.array(face_resized)
    test_image = test_image.astype('float64')
    test_image /= 255

    if num_channel==1:
        if K.image_dim_ordering()=='th':
            test_image= np.expand_dims(test_image, axis=0)
            test_image= np.expand_dims(test_image, axis=0)
        else:
            test_image= np.expand_dims(test_image, axis=3) 
            test_image= np.expand_dims(test_image, axis=0)
    else:
        if K.image_dim_ordering()=='th':
            test_image=np.rollaxis(test_image,2,0)
            test_image= np.expand_dims(test_image, axis=0)
        else:
            test_image= np.expand_dims(test_image, axis=0)
    return test_image

<h2> function for load model, image processing, reshape, dan prediksi <h2>

In [5]:
def process_image(inImg):
        #=============LDA===================
        with open ('model/LDA_mlp.pkl', 'rb') as Rpca:
            model_1= pickle.load(Rpca)
        
        with open ('model/model_LDA_mlp.pkl', 'rb') as Rca:
            model_2= pickle.load(Rca)
         #==============PCA   
#         with open ('model/PCA_mlp.pkl', 'rb') as Rpca:
#             model_1= pickle.load(Rpca)
        
#         with open ('model/model_PCA_mlp.pkl', 'rb') as Rca:
#             model_2= pickle.load(Rca)
        
        
        RESIZE_FACTOR = 4
        global names
        frame = cv2.flip(inImg,1)
        

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        hist= cv2.equalizeHist(gray)
        gray_resized=cv2.resize(hist,(160,130))
        

#         deteksi = model.predict(test_image)
        cascPath = "haarcascade/haarcascade_frontalface_default.xml"

        face_cascade = cv2.CascadeClassifier(cascPath)
        faces = face_cascade.detectMultiScale(
                gray_resized,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30),
                flags=cv2.CASCADE_SCALE_IMAGE
                )
        nama_prediksi = []
        for i in range(len(faces)):
            face_i = faces[i]
            x = face_i[0] * RESIZE_FACTOR
            y = face_i[1] * RESIZE_FACTOR
            w = face_i[2] * RESIZE_FACTOR
            h = face_i[3] * RESIZE_FACTOR
            face = gray[y:y+h, x:x+w]
            
            test_image=face1(face)
            shape = test_image.shape
            test_image = test_image.reshape((shape[0], shape[1] * shape[2]*shape[3]))
            test_image.shape
            hasil_pca=model_1.transform(test_image)
            hasil_predik= model_2.predict(hasil_pca)[0]
                
            nama = from_class_to_label(hasil_predik)
            kn='unknown'
            if nama==kn:
                salah = 'Tidak Diketahui'
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 0, 255), 3)
                cv2.putText(frame,salah, (x-10, y-10), cv2.FONT_HERSHEY_PLAIN,1,(0, 255, 0))
                
            else:
                
                cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 3)
                cv2.putText(frame, (nama), (x-10, y-10), cv2.FONT_HERSHEY_PLAIN,1,(0, 255, 0))
            nama_prediksi.append(nama)
        return (frame, nama_prediksi)

In [6]:
show_video()